**[3] DISPLAY Village menu with predicted prices using ANY ML algo:**
>IF

7.1 Temp is < 45 deg. Fahrenheit (note API returns Kelvin, convert to F)  

7.2 It will snow or get moderate or heavy rain

7.3 Village is busier than usual
>THEN

Price should be more than lowest local price
>ELSE

Price should be lowest local price

In [22]:
import requests

In [23]:
GOOGLE_API_KEY = "Your_GOOGLE_PLACES_API_key"
WEATHER_API_KEY = 'Your_OPENWEATHER_API_key'
place_id = 'ChIJPYSDLXWBwokRHLcHIl02Kh8'

In [24]:
def get_coordinates(place_id):
    place_details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=geometry&key={GOOGLE_API_KEY}"
    try:
        response = requests.get(place_details_url)
        response.raise_for_status()
        data = response.json()

        if 'result' in data and 'geometry' in data['result']:
            location = data['result']['geometry']['location']
            return location['lat'], location['lng']
        else:
            print("Failed to fetch coordinates.")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching place details: {e}")
        return None, None

In [25]:
def get_nearby_restaurants(lat, lng):
    places_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=500&types=restaurant&key={GOOGLE_API_KEY}"
    try:
        response = requests.get(places_url)
        response.raise_for_status()
        data = response.json()

        restaurant_prices = []
        if 'results' in data:
            for restaurant in data['results']:
                # Extract price level from the nearby restaurants (if available)
                price_level = restaurant.get('price_level', None)
                if price_level is not None:
                    # Google Places API uses a scale: 0 = free, 1 = inexpensive, 2 = moderate, 3 = expensive, 4 = very expensive
                    restaurant_prices.append(price_level)
        return restaurant_prices
    except requests.exceptions.RequestException as e:
        print(f"Error fetching nearby places: {e}")
        return []

In [26]:
def calculate_local_price():
    lat, lng = get_coordinates(place_id)

    if lat and lng:
        nearby_restaurants = get_nearby_restaurants(lat, lng)

        if nearby_restaurants:
            avg_price_level = sum(nearby_restaurants) / len(nearby_restaurants)
            print(f"Average price level of nearby restaurants: {avg_price_level:.2f}")

            if avg_price_level <= 1:
                local_price = 100
            elif avg_price_level == 2:
                local_price = 150
            elif avg_price_level == 3:
                local_price = 200
            else:
                local_price = 250

            print(f"Estimated local price for a meal: ${local_price}")
            return local_price
        else:
            print("No nearby restaurant data available.")
            return 100
    else:
        print("Unable to fetch coordinates.")
        return 100

local_price = calculate_local_price()

Average price level of nearby restaurants: 1.44
Estimated local price for a meal: $250


In [27]:
def get_weather(lat, lng):
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lng}&units=metric&appid={WEATHER_API_KEY}"
    try:
        response = requests.get(weather_url)
        response.raise_for_status()
        data = response.json()

        temperature = data['main']['temp']
        rain = data.get('rain', {}).get('1h', 0)
        snow = data.get('snow', {}).get('1h', 0)

        temperature_fahrenheit = (temperature * 9/5) + 32

        return temperature_fahrenheit, rain, snow
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None, None, None

In [28]:
def is_busier_than_usual():
    return True

def predict_price():
    lat, lng = get_coordinates(place_id)

    if lat and lng:
        temperature, rain, snow = get_weather(lat, lng)

        if temperature is None:
            print("Could not fetch temperature data.")
            return None

        print(f"Temperature: {temperature}°F")
        print(f"Rain Volume (last hour): {rain} mm")
        print(f"Snow Volume (last hour): {snow} mm")

        local_price = calculate_local_price()
        print(f"Local Price: {local_price}")

        is_cold = temperature < 45
        is_rain_or_snow = rain > 0 or snow > 0
        is_busy = is_busier_than_usual()

        if is_cold or is_rain_or_snow or is_busy:
            adjusted_price = local_price * 1.2
            print("Conditions met for higher pricing.")
        else:
            adjusted_price = local_price
            print("Conditions not met, keeping price at lowest.")

        print(f"Predicted Price: {adjusted_price:.2f}")
        return adjusted_price
    else:
        print("Unable to fetch location or weather data.")
        return None

predicted_price = predict_price()

Temperature: 40.370000000000005°F
Rain Volume (last hour): 4.86 mm
Snow Volume (last hour): 0 mm
Average price level of nearby restaurants: 1.44
Estimated local price for a meal: $250
Local Price: 250
Conditions met for higher pricing.
Predicted Price: 300.00
